## Ideas for statistics


#### 3 types of authorship model scores (full, 1/n, 1/position)
* Mean Scores across years (x)
* Max Scores across year & Year of Max ( )
* Average value in last 7 years  ( )
* Most productive co-author (x)

#### Remaining Analysis from 1/position
* "RI" Conf sub-score (X)
* "Top Graphics","Top Vision", "Top Robotics", "Top ML", "Other" sub-scores (x)
* Average number of authors (x)
* Average Author Position (x)
* Average & Median "quality" of collab ( )
* Current Affiliation (x)
* Total number of collab ( )
* Top 3 collabs (x)
* Top 3 conferences from generated value ( )
* Career length (x)
* Number of collabs w/ more than 4 papers

#### Advanced Stats from 1/n
* 5 unlabeled variants of plus-minus (w/ intercept) (x)
* 5 unlabeled variants of plius-minus (w/o intercept) (x)

#### NSF Data
* Total number of grants (x)
* Total grant money (x)
* fractional grant money (x)
* grant money of collabs ( )

In [ ]:
import os
import sys
import fnmatch
import numpy as np
import pandas as pd
import json
import gzip
import pickle
import csv
import scipy.sparse
Xauth = None
from collections import defaultdict
import matplotlib.pyplot as plt

In [ ]:
# setup the update to work despite the broken scipy documentation
try:
    a = scipy.sparse.dok_matrix((10,10))
    a.update({(0,0):1.0})
    scipy.sparse.dok_matrix.my_update = scipy.sparse.dok_matrix.update
except:
    a = scipy.sparse.dok_matrix((10,10))
    a._update({(0,0):1.0})
    scipy.sparse.dok_matrix.my_update = scipy.sparse.dok_matrix._update

In [ ]:
with gzip.open('useful_venue_list.pkl.gz','rb') as fp:
    all_venues = pickle.load(fp)
with gzip.open('useful_authors_list.pkl.gz','rb') as fp:
    all_authors = pickle.load(fp)
with gzip.open('useful_papers.pkl.gz','rb') as fp:
    all_papers = pickle.load(fp)


In [ ]:

min_year = all_papers[0][6]
max_year = all_papers[-1][6]
span_years = max_year - min_year + 1
print(min_year,max_year,span_years)
conf_idx = {v:i for i,v in enumerate(all_venues)}
name_idx = {v:i for i,v in enumerate(all_authors)}
n_confs = len(all_venues)
n_auths = len(all_authors)
n_papers = len(all_papers)
print(n_confs,n_auths,n_papers)

In [ ]:
scoreV = {}
clf =  np.load('clf_gold.pkl.npy')
years_per_conf = clf.shape[0]//n_confs
YEAR_BLOCKS = span_years//years_per_conf
clf[2323]


In [ ]:

for FI in [False,True]:
    scoreV['_apm' + str(FI)] = np.load('apm'+str(FI) + '.npy')
    scoreV['pw_apm' + str(FI)] = np.load('pwapm'+str(FI) + '.npy')
    scoreV['pweff_apm' + str(FI)] = np.load('pweffapm'+str(FI) + '.npy')
    scoreV['pwunk_apm' + str(FI)] = np.load('pwunkapm'+str(FI) + '.npy')
    print(scoreV['pwunk_apm' + str(FI)].shape)


In [ ]:
try:
    import gzip
    import pickle
    with gzip.open('scoresV2.pkl.gz','rb') as fp:
        scoreV = pickle.load(fp)
except:
    print('failed!')
PROCESS_DATA = len(scoreV) < 13
print(scoreV['pwunk_apm' + str(FI)].shape)


In [ ]:
#PROCESS_DATA = True

In [ ]:
with open('top_ri_Metric.json','rt') as fp:
    interesting_set = set(json.load(fp))
print(len(interesting_set))
curious_names = ['Xiaolong Wang 0004','Judy Hoffman','Paris Siminelakis','Roie Levin','Leonid Keselman',
                 'Nicholas Rhinehart','Vincent Sitzmann','Siddharth Ancha','Xingyu Lin',
                 'Humphrey Hu','Aditya Dhawale','Nick Gisolfi','Andrey Kurenkov','Micah Corah',
                 'David F. Fouhey','Chelsea Finn','Akshara Rai','Ankit Bhatia','Xuning Yang',
                 'Lerrel Pinto','Graeme Best','Alexander Spitzer','Roberto Shu','Amir Abboud',
                 'Justin Johnson','Kumar Shaurya Shankar','Ellen A. Cappo',
                 'Amir Roshan Zamir','Dominik Peters','Jonathan T. Barron','Dorsa Sadigh','Derek Hoiem','Vaggos Chatziafratis',
                 'Brian Okorn','David Held']
#interesting_set = set(curious_names)

In [ ]:
cmu_uni = pd.read_csv('other_ranks/cmu_faculty.csv')
cmu_uni = cmu_uni.fillna('Other')
cmu_uni = cmu_uni[cmu_uni.dept == 'RI']
uni_names = set(list(cmu_uni.name))
print(len(uni_names))
conf_counts = {}
conf_counts_value = {}

#interesting_set = uni_names

for paper in all_papers:
    tag,title, authors, venue, pages, startPage,year,volume,number,url,publtype,eb_toofew,eb_skip = paper
    if year < 2000:
        continue
    n = len(authors)
    for a in authors:
        if a in uni_names:
            conf_counts[venue] = 1/n + conf_counts.get(venue,0)
            conf_counts_value[venue] = clf[years_per_conf*(conf_idx[venue]) + (year-min_year)//YEAR_BLOCKS]/n + conf_counts_value.get(venue,0)
conf_counts_value = {k: v/conf_counts[k] for k,v in conf_counts_value.items()}
ri_fav_confs = [(conf_counts[_[1]]*conf_counts_value[_[1]],_[1],conf_counts[_[1]],conf_counts_value[_[1]]) for _ in sorted([(v,k) for k,v in conf_counts.items() if v > 0],reverse=True)]

In [ ]:
ri_confs = [_[1] for _ in sorted(ri_fav_confs,reverse=True) if _[-2] >= 1.25]
#confs_to_filter =['ICRA','IROS','Robotics: Science and Systems']
ri_confs

In [ ]:
if PROCESS_DATA:
    auth_years = np.ones((n_auths,2)) * np.array([3000,1000]) 
    for paper in all_papers:
        tag,title, authors, venue, pages, startPage,year,volume,number,url,publtype,eb_toofew,eb_skip = paper
        for a in authors:
            i = name_idx[a]
            auth_years[i,0] = min(auth_years[i,0],year)
            auth_years[i,1] = max(auth_years[i,1],year)
    working_years = (auth_years[:,1] - auth_years[:,0]+1)
    scoreV['working_years'] = working_years
    scoreV['auth_years'] = auth_years


In [ ]:
if PROCESS_DATA:
    valid_ns = set()
    for paper in all_papers:
        tag,title, authors, venue, pages, startPage,year,volume,number,url,publtype,eb_toofew,eb_skip = paper
        n = len(authors)
        valid_ns.add(n)


In [ ]:
conf_types = {
        'RI': ri_confs,
        'ML':['NIPS','ICML','AAAI','AISTATS','IJCAI','UAI','CoRL','ICLR'],
        'CV':['CVPR','ICCV','ECCV','IEEE Trans. Pattern Anal. Mach. Intell.','FGR','Int. J. Comput. Vis.','WACV','BMVC'],
        'ROB':['HRI','Int. J. Robotics Res.','Robotics: Science and Systems','Humanoids','WAFR','IROS','ICRA','FSR','ISER','ISRR','AAMAS','IEEE Robotics Autom. Lett.','IEEE Trans. Robotics and Automation'],
        'GR':['ACM Trans. Graph.','Comput. Graph. Forum','SIGGRAPH','SIGGRAPH Asia','Symposium on Computer Animation','AISTATS','IJCAI','UAI'],
             }

In [ ]:
am_types = ['full','1/n','1/i']
year_filters = [1970,1990,2000,2010]

In [ ]:
if PROCESS_DATA:
    confTypeN = len(conf_types)+1
    YearConf = scipy.sparse.lil_matrix((n_confs*years_per_conf,years_per_conf*confTypeN))
    for i in range(years_per_conf):
        year_filter = np.zeros_like(clf).reshape((-1,years_per_conf))
        year_filter[:,i] = 1
        YearConf[:,i*confTypeN] = (clf * year_filter.reshape(clf.shape))[:,np.newaxis]
        j = 1
        for f_type, f_confs in conf_types.items():
            year_filter = np.zeros_like(clf).reshape((-1,years_per_conf))
            for conf in f_confs:
                year_filter[conf_idx[conf],i] = 1
            YearConf[:,i*confTypeN+j] = (clf * year_filter.reshape(clf.shape))[:,np.newaxis]
            j+=1
    YearConf = scipy.sparse.csr_matrix(YearConf)

In [ ]:
import scipy.sparse
import gc
if PROCESS_DATA:
    for amt in am_types:

        per_author_val = {}

        if amt == 'full':
            for n in valid_ns:
                author_scores = np.ones(n)
                per_author_val[n] = author_scores
        elif amt == '1/n':
            for n in valid_ns:
                author_scores = (np.ones(n))
                per_author_val[n] = author_scores/author_scores.sum()
        elif amt == '1/i':
            for n in valid_ns:
                author_scores = 1/(np.arange(n)+1)
                per_author_val[n] = author_scores/author_scores.sum()
        else:
            raise

        count_vecs = {}
        paper_vecs = []
        for paper in all_papers:
            tag,title, authors, venue, pages, startPage,year,volume,number,url,publtype,eb_toofew,eb_skip = paper
            n = len(authors)
            j = years_per_conf*conf_idx[venue] + (year-min_year)//YEAR_BLOCKS

            author_scores = per_author_val[n]
            paper_vecs.append([(name_idx[a],j,v) for a,v in zip(authors,author_scores)])

        Xauth = scipy.sparse.dok_matrix((n_auths,years_per_conf*n_confs))
        xdict = {}

        for paper_vec in paper_vecs:
            for i,j,v in paper_vec:
                xdict[(i,j)] = v + xdict.get((i,j),0)

        Xauth.my_update(xdict)

        Xauth = scipy.sparse.csr_matrix(Xauth)



        scoreV[amt] = Xauth @ YearConf


        paper_vec = []
        xdict = {}
        gc.collect()

In [ ]:
if PROCESS_DATA:
    import gzip
    import pickle
    with gzip.open('scoresV2.pkl.gz','wb') as fp:
        pickle.dump(scoreV,fp)

In [ ]:
print(len(scoreV))#,years_per_conf

In [ ]:
import scipy.ndimage
for am in am_types:
    scores = np.array(scoreV[am].todense()).reshape((n_auths,years_per_conf,-1)).astype(np.float32)
    scores = np.transpose(scores,(0,2,1))
    smooth_kernel = scipy.ndimage.gaussian_filter1d(np.identity(years_per_conf,np.float32),1)
    scores = scores @ smooth_kernel
    scoreV[am] = scores

In [ ]:
sTypes = ['Full'] + [k for k,v in conf_types.items()]

In [ ]:
scores.dtype,scores.nbytes,gc.collect()

In [ ]:
auth_years = scoreV['auth_years']
working_years = scoreV['working_years']

total_scores = scoreV['1/i'][:,sTypes.index('Full')].sum(1)
ri_scores = scoreV['1/i'][:,sTypes.index('RI')].sum(1)
ri_eff_scores = ri_scores/working_years#,np.maximum(auth_years[:,1]-2000,1))

ri_scores_max = scoreV['1/i'][:,sTypes.index('RI')].max(1)
ri_scores_max_yr = np.argmax(scoreV['1/n'][:,sTypes.index('RI')],axis=1)*YEAR_BLOCKS + min_year


In [ ]:
#best_idx = np.argsort(total_scores)[::-1]
#for k in range(10):
#    idx = best_idx[k]
#    print('{:30s}\t{:.2f}'.format(all_authors[idx],total_scores[idx]))

In [ ]:
interesting_set = set()
with open('top_ri_Metric.json','rt') as fp:
    interesting_set = set(json.load(fp))
    print(len(interesting_set))
#interesting_set.add('Jeff Clune')
interesting_set = set([_ for _ in uni_names if _ in name_idx])
print(len(interesting_set))
interesting_set.add('Dinesh Jayaraman')

In [ ]:
pot_cand_df = pd.read_csv('pot_export.csv',index_col=0)
interesting_set = set(pot_cand_df.Author)

In [ ]:
Directors = ['Takeo Kanade','Martial Hebert','Matthew T. Mason']
labels = ['TK','MH','MM','RD']
for n in Directors:
    interesting_set.add(n)

In [ ]:
best_idx = np.argsort(ri_scores_max * (working_years > 5))[::-1]
for k in range(1000):
    idx = best_idx[k]
    #interesting_set.add(all_authors[idx])
    print('{}\t{:30s}\t{:.2f}\t{:.2f}\t{:.2f}\t{:d}\t{:d}'.format(k,all_authors[idx],ri_scores_max[idx],ri_eff_scores[idx],ri_scores[idx],ri_scores_max_yr[idx],int(working_years[idx])))

In [ ]:
best_idx = np.argsort(ri_eff_scores * (working_years > 5))[::-1]
for k in range(150):
    idx = best_idx[k]
    #interesting_set.add(all_authors[idx])
    print('{}\t{:30s}\t{:.2f}\t{:.2f}\t{:d}'.format(k,all_authors[idx],ri_eff_scores[idx],ri_scores[idx],int(working_years[idx])))

In [ ]:
best_idx = np.argsort(ri_scores)[::-1]
for k in range(150):
    idx = best_idx[k]
    #interesting_set.add(all_authors[idx])

    print('{}\t{:30s}\t{:.2f}\t{:.2f}\t{:d}'.format(k,all_authors[idx],ri_scores[idx],ri_scores[idx]/total_scores[idx],int(auth_years[idx,0])))

In [ ]:
best_idx = np.argsort(total_scores)[::-1]
for k in range(150):
    idx = best_idx[k]
    #interesting_set.add(all_authors[idx])

    print('{}\t{:30s}\t{:.2f}\t{:.2f}\t{:d}'.format(k,all_authors[idx],ri_scores[idx],ri_scores[idx]/total_scores[idx],int(auth_years[idx,0])))

In [ ]:
#with open('top_ri3.json','wt') as fp:
#    json.dump(sorted(list(interesting_set)),fp,sort_keys=True,indent=4, separators=(',', ': '))

In [ ]:
faculty_affil = pd.read_csv('faculty-affiliations.csv')
year_span = (auth_years[:,1] - auth_years[:,0]) + 1
faculty_lookup = {_[1]:_[2] for _ in faculty_affil.itertuples()}
faculty_lookup['Reid G. Simmons'] = 'Carnegie Mellon University'
faculty_lookup['Sebastian Scherer'] = 'Carnegie Mellon University'
faculty_lookup['Jeff G. Schneider'] = 'Carnegie Mellon University'
for row in cmu_uni.itertuples():
    faculty_lookup[row[1]] = 'Carnegie Mellon University'

## Author affiliated stats

In [ ]:
coauthors  = defaultdict(lambda: defaultdict(int))
coauthors_frac  = defaultdict(lambda: defaultdict(float))
coauthors_num  = defaultdict(list)
author_pos  = defaultdict(list)
conf_paper_frac  =  defaultdict(lambda: defaultdict(int))
paper_num  = defaultdict(int)
paper_frac  = defaultdict(float)


In [ ]:
for paper in all_papers:
    tag,title, authors, venue, pages, startPage,year,volume,number,url,publtype,eb_toofew,eb_skip = paper
    n = len(authors)
    parse = False
    for a in authors:
        if a in interesting_set:
            parse = True
            break
    if parse:
        for i,a in enumerate(authors):
            if a in interesting_set:
                conf_paper_frac[a][venue] += 1/n
                coauthors_num[a].append(n)
                author_pos[a].append((i+1)/n)
                paper_num[a] += 1
                paper_frac[a] += 1/n
                for a2 in authors:
                    if a2 == a:
                        continue
                    coauthors[a][a2] += 1
                    coauthors_frac[a][a2] += 1/n
                    

In [ ]:
sorted([(v,k) for k,v in coauthors['Martial Hebert'].items()],reverse=True)

In [ ]:
np.array(coauthors_num['Martial Hebert']).mean()

In [ ]:
pr_full = pickle.load(open('new_pagerank_people.pkl','rb'))
pr_ri = pickle.load(open('new_pagerank_people_ri.pkl','rb'))
pr_full /= pr_full.max()
pr_ri /= pr_ri.max()
print(pr_ri.shape,pr_full.shape)

# Build it

In [ ]:
from unidecode import unidecode
author_results = defaultdict(dict)
for name in interesting_set:
    idx = name_idx[name]
    author_results[name]['Affiliation'] = faculty_lookup.get(name,'Unknown')
    author_results[name]['Years'] = scoreV['working_years'][idx]

In [ ]:
scoreV['1/i_total_1970'] = scoreV['1/i'][:,sTypes.index('Full')].sum(1)
scoreV['1/i_RI_1970'] = scoreV['1/i'][:,sTypes.index('RI')].sum(1)
for sub in ['ROB','CV','GR','ML']:
    den = scoreV['1/i_{}_1970'.format(sub)] = scoreV['1/i'][:,sTypes.index(sub)].sum(1)
scoreV['full_total_1970'] = scoreV['full'][:,sTypes.index('Full')].sum(1)
scoreV['1/n_total_1970'] = scoreV['1/n'][:,sTypes.index('Full')].sum(1)


scoreV['1/n_max'] = scoreV['1/n'][:,sTypes.index('Full')].max(1)
scoreV['1/n_max_yr'] = np.argmax(scoreV['1/n'][:,sTypes.index('Full')],axis=1)*YEAR_BLOCKS+min_year

scoreV['1/i_max'] = scoreV['1/i'][:,sTypes.index('Full')].max(1)
scoreV['1/i_max_yr'] = np.argmax(scoreV['1/i'][:,sTypes.index('Full')],axis=1)*YEAR_BLOCKS+min_year

scoreV['full_max'] = scoreV['full'][:,sTypes.index('Full')].max(1)
scoreV['full_max_yr'] = np.argmax(scoreV['full'][:,sTypes.index('Full')],axis=1)*YEAR_BLOCKS+min_year

In [ ]:
for k,v in scoreV.items():
    print(k,v.shape)

In [ ]:
v = scoreV['1/i_total_1970']/(np.maximum(2,working_years.astype(np.float32)))
ratio_v = np.maximum(1e-3,scoreV['1/i_RI_1970'])/np.maximum(1e-3,scoreV['1/i_total_1970'])
v *= np.sqrt(ratio_v)
v *= 1/np.log(np.maximum(2,working_years.astype(np.float32)))
tv = np.zeros_like(scoreV['1/i_total_1970'])
for n in ['_apmFalse', '_apmTrue',  'pw_apmFalse', 'pw_apmTrue', 'pweff_apmFalse', 'pweff_apmTrue', 'pwunk_apmFalse','pwunk_apmTrue']:
    print(n,scoreV[n].shape)
    tv +=  scoreV[n]
v *= np.log(np.maximum(tv,10))
v *= np.log(np.maximum(np.exp(1),scoreV['full'][:,sTypes.index('Full')].sum(1)))
v *= np.log(scoreV['1/i_max_yr']-1965)
v = np.nan_to_num(v)
meta_metric = v

In [ ]:
for name in interesting_set:
    idx = name_idx[name]
    author_results[name]['MetaMetric'] = v[idx]
    author_results[name]['Score (1/pos)'] = scoreV['1/i_total_1970'][idx]
    author_results[name]['RIScore'] = scoreV['1/i_RI_1970'][idx]
    author_results[name]['RI %'] = scoreV['1/i_RI_1970'][idx]/scoreV['1/i_total_1970'][idx]

    sum_v = 0
    for sub in ['ROB','CV','GR','ML']:
        den = scoreV['1/i_{}_1970'.format(sub)][idx]
        den = den if den != 0.0 else 1
        author_results[name][sub + ' %'] = den/scoreV['1/i_total_1970'][idx]
        sum_v += den
    author_results[name]['Other %'] = max(0,scoreV['1/i_total_1970'][idx] - sum_v)/scoreV['1/i_total_1970'][idx]
    
    author_results[name]['From'] = scoreV['auth_years'][idx][0]
    author_results[name]['Until'] = scoreV['auth_years'][idx][1]

    author_results[name]['YearlyScore (1/pos)'] = scoreV['1/i_total_1970'][idx]/scoreV['working_years'][idx]
    author_results[name]['YearlyRIScore'] = scoreV['1/i_RI_1970'][idx]/scoreV['working_years'][idx]

new_set = set()

for name in interesting_set:
    idx = name_idx[name]

    author_results[name]['avgCoauthor'] = np.array(coauthors_num[name]).mean()
    colabs = sorted([(v,k) for k,v in coauthors_frac[name].items()],reverse=True)
    fam_colab = sorted([(v*scoreV['1/i_total_1970'][name_idx[k]],k) for k,v in coauthors_frac[name].items()],reverse=True)
    
    freq_colabs = sorted([(v,k) for k,v in coauthors[name].items() if v >= 4],reverse=True)
    if len(colabs) > 0:
        author_results[name]['mostCoauthorName'] = unidecode(colabs[0][1])
        author_results[name]['mostCoauthorTimes'] = colabs[0][0]
        new_set.add(colabs[0][1])
    else:
        author_results[name]['mostCoauthorName'] = ''
        author_results[name]['mostCoauthorTimes'] = 0

    if len(fam_colab) > 0:
        author_results[name]['famCoauthorName'] = unidecode(fam_colab[0][1])
        new_set.add(fam_colab[0][1])
    else:
        author_results[name]['famCoauthorName'] = ''

    author_results[name]['authorPosition%'] = np.array(author_pos[name]).mean()
    author_results[name]['totalCoauth'] = len(colabs)
    author_results[name]['freqCoauth (> 3 papers)'] = len(freq_colabs)
    author_results[name]['famCoauthFrac'] = sum([_[0] for _ in fam_colab])
    author_results[name]['totalCoauthFrac'] = sum([_[0] for _ in colabs])
    
    author_results[name]['mostPaperConf'] = sorted([(v,k) for k,v in conf_paper_frac[name].items()],reverse=True)[0][1]
    author_results[name]['venuesPublishedIn'] = len(conf_paper_frac[name].items())
    author_results[name]['pageRank'] = pr_full[idx]
    author_results[name]['pageRankRI'] = pr_ri[idx]
    
    
    author_results[name]['numPapers'] = paper_num[name]
    author_results[name]['numPapersFrac'] = paper_frac[name]


    author_results[name]['YearlyScore (1/n)'] = scoreV['1/n_total_1970'][idx]/scoreV['working_years'][idx]
    author_results[name]['YearlyScore (Full)'] = scoreV['full_total_1970'][idx]/scoreV['working_years'][idx]
    author_results[name]['Score (1/n)'] = scoreV['1/n_total_1970'][idx]
    author_results[name]['Score (Full)'] = scoreV['full_total_1970'][idx]
    
    author_results[name]['MaxScore'] = scoreV['1/i_max'][idx]
    author_results[name]['MaxScore (1/n)'] = scoreV['1/n_max'][idx]
    author_results[name]['MaxScore (Full)'] = scoreV['full_max'][idx]
    
    author_results[name]['BestYear'] = scoreV['1/i_max_yr'][idx]
    author_results[name]['BestYear (1/n)'] = scoreV['1/n_max_yr'][idx]
    author_results[name]['BestYear (Full)'] = scoreV['full_max_yr'][idx]
    
    for i,n in enumerate(['_apmFalse', '_apmTrue',  'pw_apmFalse', 'pw_apmTrue', 'pweff_apmFalse', 'pweff_apmTrue', 'pwunk_apmFalse','pwunk_apmTrue']):
        author_results[name]['Adv'+str(i+1)] =  scoreV[n][idx]


In [ ]:
results_list = []
for k,v in author_results.items():
    v['Name'] = k
    results_list.append(v)
def_order = list(author_results[list(author_results.keys())[0]].keys())
df_results = pd.DataFrame(results_list)[def_order].set_index('Name')

In [ ]:
df_results = df_results.fillna(0.0)
df_results


In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
vecs = ss.fit_transform(df_results._get_numeric_data())
pca = PCA(n_components=5,whiten=True)
res = pca.fit_transform(vecs)
pca.explained_variance_
for i in range(5):
    df_results['pca'+str(i)] = res[:,i]

In [ ]:
df_results_out = df_results[~df_results.index.isin(Directors)]
pot_name = pot_cand_df.set_index('Author')
pot_name.index = pot_name.index.rename('Name')
df_results_out2 = df_results_out.join(pot_name[pot_name.columns.difference(df_results_out.columns)])
adv_totals = df_results_out2[[_ for _ in df_results_out2.columns if 'Adv' in _ and 'NSF' not in _]].sum(1)
df_results_out2['AdvTotal'] = adv_totals


In [ ]:
df_results_out2['MetaMetric'] = df_results_out2['RIScore']  + np.sqrt(df_results_out2['eTotal'])
df_results_out2['MetaMetric'] = df_results_out2['MetaMetric'] + 0.1* (df_results_out2['Score (1/pos)']-df_results_out2['RIScore'])
df_results_out2['MetaMetric'] = df_results_out2['MetaMetric'] + 0.1 * adv_totals


In [ ]:
df_results_out2 = df_results_out2.sort_values('MetaMetric',0,False)
#df_results_out2[[_ for _ in df_results_out2.columns if '%' in _ and 'author' not in _ and 'Other' not in _]].sum(1)
df_results_out2#[df_results_out2.MetaMetric > 0]
df_out2 = df_results_out2[(df_results_out2['RI %'] > 0.1) & (df_results_out2.MetaMetric > 0) & (df_results_out2['Score (1/pos)'] > 5)]
df_out2#.loc['Júlia Borràs Sol']

In [ ]:
[_ for _ in interesting_set if 'Sol' in _]

In [ ]:
df_out2.round(2).to_excel('dump.xlsx')
raise

In [ ]:
new_new = set()
for name in set(list(interesting_set) +list(new_set)):
    idx = name_idx[name]
    if scoreV['working_years'][idx] < 10:
        pass
    else:
        new_new.add(name)

In [ ]:

spec_vectors = [res[list(df_results.index).index(_)] for _ in Directors]
spec_vectors.append(np.array(spec_vectors).mean(0))
spec_vectors = np.array(spec_vectors)

In [ ]:
df_results.iloc[np.argmax(res[:,0])]

In [ ]:
plt.scatter(res[:,0],res[:,1])
DIR_IDX = df_results.index.isin(Directors)
plt.scatter(spec_vectors[:,0],spec_vectors[:,1])
#df_results.iloc[6737]
#np.where(res[:,0] > 7) # 2004, 6737

In [ ]:
from scipy.spatial.distance import cdist,squareform
dists = cdist(spec_vectors,res,'euclidean')
for l,d in zip(labels,dists):
    df_results['euc' + l] = d
dists = cdist(spec_vectors,res,'cosine')
for l,d in zip(labels,dists):
    df_results['cos' + l] = d
df_results['MetaMetric'] *= (1/np.log(np.maximum(0.07,df_results['cosRD'])*50))

In [ ]:
df_results.sort_values('cosRD').to_csv('profile_results5.csv')
df_results.corr('spearman').to_csv('corr.csv')

In [ ]:
df_results.sort_values('NSF YearlyAward',0,False)

In [ ]:
df_results.columns

In [ ]:
plt.style.use('fivethirtyeight')

name_plot = ['David P. Woodruff',"Ryan O'Donnell",'Anupam Gupta']
for name in name_plot:
    plt.plot(np.arange(min_year,max_year,YEAR_BLOCKS),scoreV['1/i'][name_idx[name],sTypes.index('Full')],label=name.split()[0])


plt.legend()
plt.xlim(left=2000)

In [ ]:
#=D3*SQRT(F3)*(1/LOG(C3))*LOG(MAX(10,SUM(AN3:AU3)))*MAX(1,LOG(AE3))*(1/LOG(MAX(0.07,BN3)*50))*LOG(AM3-1965)

In [ ]:
interesting_set = set()
with open('top_ri3.json','rt') as fp:
    interesting_set = set(json.load(fp))
    print(len(interesting_set))
#interesting_set.add('Jeff Clune')


In [ ]:
best_v = np.argsort(meta_metric)[::-1]
len(v)

In [ ]:
print(len(interesting_set))
beST_N = 15000
for i in range(beST_N):
    idx = best_v[i]
    if working_years[idx] >= 10:
        interesting_set.add(all_authors[idx])
    #print(idx,all_authors[idx],v[idx],ratio_v[idx])
print(meta_metric[best_v[beST_N]],meta_metric[name_idx['Odest Chadwicke Jenkins']],meta_metric[name_idx['Jeff Clune']])
print(len(interesting_set))

In [ ]:
with open('top_ri_Metric.json','wt') as fp:
    json.dump(sorted(list(interesting_set)),fp,sort_keys=True,indent=4, separators=(',', ': '))